In [21]:
%run input/Format.ipynb
import ROOT as root
from array import array
from math import sin
from ROOT import gRandom
from ROOT import TFile
from ROOT import TPad, TLegend
root.gErrorIgnoreLevel = root.kFatal
%jsroot on

In [22]:
iOption0 = 0

In [23]:
isigma = 2
N_files = 1
N_centr = 6
N_hists = 4
N_layers = 6
N_ilyaers = 1
is_sigmalised = 0
is_e = 1
iOption1 = 0
centralities = ["1 layer pos","1 layer neg","2 layer pos","2 layer neg","3 layer pos","3 layer neg","20-40%","40-60%","60-80%","80-93%"]
centrality = [1,1,2,2,3,3,4,4,5,5]
layer_bins = [35,36,40,41,47,48,52,53,59,60,64,65]
layer_bins = [46,46,48,48,50,50,52,52,54,54,56,56]
colors=[1,2,4,root.kGreen+2,root.kMagenta,3]
file_path="input/rg/"
file_names="my-24th_QA_19192_80runs_rg"
hist_select_3D_names = ["dphi_hist","dthe_hist","sdphi_hist","sdthe_hist"]
if is_e == 1: hist_select_3D_names = ["dphi_hist_el","dthe_hist_el","sdphi_hist_el","sdthe_hist_el"]

In [24]:
hists_emc_read = []

for iFile in range(N_files):
    print(file_path+file_names+f"{iFile}.root")
    infile = root.TFile.Open(file_path+file_names+f"{iFile}.root", "read")
    hists_emc_read0 = []
    for icentr in range(N_centr):
        hist_select_3D0 = []
        for ihist in range(len(hist_select_3D_names)):
            hist_select_3D0.append(infile.Get(hist_select_3D_names[ihist]+f"_{icentr}"))
            hist_select_3D0[-1].SetDirectory(root.nullptr)
        hists_emc_read0.append(hist_select_3D0)
    hists_emc_read.append(hists_emc_read0)

infile.Close()

input/rg/my-24th_QA_19192_80runs_rg0.root


In [25]:
pt = array( 'd' )
pt_boarders = array( 'd', [0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0,1.4,2.0,2.8,4.9])
pt_err = array( 'd' )
N_pt = len(pt_boarders) - 1
for i in range(N_pt):
    pt.append( (pt_boarders[i]+pt_boarders[i+1])/2)
    pt_err.append((pt_boarders[i+1]-pt_boarders[i])/2)
print("pt = ", pt,"\npt_boarders = ",pt_boarders)
pt[0]=0.225

pt =  array('d', [0.25, 0.35, 0.45, 0.55, 0.6499999999999999, 0.75, 0.8500000000000001, 0.95, 1.2, 1.7, 2.4, 3.85]) 
pt_boarders =  array('d', [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.4, 2.0, 2.8, 4.9])


In [26]:
boards_tot = [[-0.099,0.099],[-0.099,0.099]]
boards_fit = [[-0.011,0.012],[-0.011,0.012]]
boards_fit2 = [[-0.03,-0.01],[0.015,0.035],[-0.09,-0.03],[0.035,0.09],[-0.06,-0.025],[0.03,0.065]]
boards_fit3 = [[-0.02,0.02],[-0.02,0.02]]
if iOption1==1:
    boards_tot = [[-5,5],[-5,5]]
    boards_fit = [[-1,1],[-1,1]]
if is_sigmalised == 1:
    boards_tot = [[-2,2],[-2,2]]
    boards_fit = [[-1,1],[-1,1]]
    boards_fit2 = [[0.6,1.5],[0.6,1.5]]
    boards_fit3 = [[-2,2],[-2,2]]

In [27]:
def emc_fitter(InHist, ilayer, seed=1, icentr=0, sigma=2, fits_mean=0, fits_sigma=0):
    N_gaus=2
    seed0 = seed
    seed=seed*N_pt*icentr*100

    hist_pt_bin = InHist.ProjectionZ("kek")

    central_bean_prev = (0.002*layer_bins[2*ilayer]-0.001-0.1)
    Mean_local =  array( 'd' )
    Sigma_local =  array( 'd' )
    Mean_local_err =  array( 'd' )
    Sigma_local_err =  array( 'd' )

    int_fg_local =  array( 'd' )
    int_bg_local =  array( 'd' )
 
    for p in range(N_pt):
        c1 = root.TCanvas(f"c1{seed+p}",f"c1{seed+p}",720,360)

        m2_name=str(icentr)+"/"+str(ilayer)+"dynamic_"+str(round(pt[p],1))

        emc_var_porj = InHist.ProjectionX(f"emc{seed+p}_{seed+p}_{icentr}_{ilayer}",layer_bins[2*ilayer],layer_bins[2*ilayer+1],hist_pt_bin.FindBin(pt_boarders[p]),hist_pt_bin.FindBin(pt_boarders[p+1])-1)
        emc_var_porj.SetLineColor(1)

        par_tot = array( 'd', (3*N_gaus+3)*[2.] )
        g_gaus= root.TF1( f'g_gaus_{seed+p}_{icentr}_{ilayer}', 'gaus',  boards_fit[int(icentr%2)][0]+central_bean_prev, boards_fit[int(icentr%2)][1]+central_bean_prev)
        g_gaus2= root.TF1( f'g_gaus_2_{seed+p}_{icentr}_{ilayer}', 'gaus',  boards_fit2[icentr][0]+central_bean_prev, boards_fit2[icentr][1]+central_bean_prev)#[0]/[2]/sqrt(2*3.14159)*exp(-0.5*pow((x-[1])/[2],2))
        g_pol = root.TF1( f'g_pol{seed+p}_{icentr}_{ilayer}', '[0]+[1]*x*(abs(x)/x-1)+[2]*x*(abs(x)/x+1)',  boards_tot[int(icentr%2)][0],boards_tot[int(icentr%2)][1])
        Format_Func(g_pol,3,2)

        total = root.TF1( f'total_pos{seed+p}_{icentr}_{ilayer}', 'gaus(0)+gaus(3)+[6]+[7]*x*(abs(x)/x-1)+[8]*x*(abs(x)/x+1)',boards_tot[int(icentr%2)][0], boards_tot[int(icentr%2)][1])
        Format_Func(total,2,6)

        g_gaus.SetParameter(1,0.)
        g_gaus.SetParameter(2,boards_fit[1][1])
        g_gaus2.SetParameter(1,0.5*(boards_fit2[icentr][0]+boards_fit2[icentr][1])+central_bean_prev)
        g_gaus2.SetParameter(2,0.005)
        g_gaus2.SetParLimits(0,1,10000)
        g_gaus2.SetParLimits(1,boards_fit2[icentr][0]+central_bean_prev, boards_fit2[icentr][1]+central_bean_prev)

        emc_var_porj.Draw()
        emc_var_porj.Fit( g_gaus, 'QR')

        if iOption0 == 1:
            Mean_local.append(g_gaus.GetParameter(1))
            Sigma_local.append(g_gaus.GetParameter(2))
            Mean_local_err.append(g_gaus.GetParError(1)+0.05*g_gaus.GetParameter(2))
            Sigma_local_err.append(g_gaus.GetParError(2)+0.05*g_gaus.GetParameter(2))
            c1.SaveAs( "output/d_delta_fits/" + m2_name + ".png" )
            continue

        emc_var_porj.Fit( g_gaus2, 'QR+' )
        emc_var_porj.Fit( g_pol, 'QR+' )

        par_gaus = g_gaus.GetParameters()
        par_gaus2 = g_gaus2.GetParameters()
        par_pol = g_pol.GetParameters()

        par_tot[0], par_tot[1], par_tot[2], par_tot[3], par_tot[4], par_tot[5] = par_gaus[0], par_gaus[1], par_gaus[2], par_gaus2[0], par_gaus2[1], par_gaus2[2]
        par_tot[6], par_tot[7], par_tot[8] = par_pol[0], par_pol[1], par_pol[1]

        total.SetParameters( par_tot )
        total.SetParLimits(1,boards_tot[int(icentr%2)][0]/10+central_bean_prev,boards_tot[int(icentr%2)][1]/10+central_bean_prev)
        total.SetParLimits(2,boards_fit[int(icentr%2)][1]/10,2)
        total.SetParLimits(3,par_gaus2[0]*0.49, par_gaus2[0]*2)
        total.SetParLimits(4,par_gaus2[1]-0.01, par_gaus2[1]+0.01)
        total.SetParLimits(5,par_gaus2[2]*0.49, par_gaus2[2]*2)
        #total.SetParLimits(3,1,1000000)
        if ilayer>-1:
            total.SetParLimits(1,boards_tot[int(icentr%2)][0],boards_tot[int(icentr%2)][1])

        
        emc_var_porj.Draw()
        emc_var_porj.Fit( total, 'QR' )

        par_tot = total.GetParameters()

        g_gaus= root.TF1( f'g_gaus_4_{seed+p}_{icentr}_{ilayer}', 'gaus',  par_tot[1]-2*par_tot[2], par_tot[1]+2*par_tot[2])
        g_gaus2= root.TF1( f'g_gaus_5_{seed+p}_{icentr}_{ilayer}', 'gaus',  par_tot[4]-2*par_tot[5], par_tot[4]+2*par_tot[5])

        Format_Func(g_gaus,3,4)
        Format_Func(g_gaus2,3,root.kGreen+2)

        par_gaus[0], par_gaus[1], par_gaus[2], par_gaus2[0], par_gaus2[1], par_gaus2[2] = par_tot[0], par_tot[1], par_tot[2], par_tot[3], par_tot[4], par_tot[5]
        par_pol[0], par_pol[1], par_pol[2] = par_tot[6], par_tot[7], par_tot[8]
        g_gaus2.SetParameters(par_gaus2)
        g_gaus.SetParameters(par_gaus)
        g_pol.SetParameters(par_pol)

        g_gaus.Draw('same')
        g_gaus2.Draw('same')
        g_pol.Draw('same')

        if iOption0 ==0:
            Mean_local.append(total.GetParameter(1))
            Sigma_local.append(total.GetParameter(2))
            Mean_local_err.append(total.GetParError(1)+0.05*total.GetParameter(2))
            Sigma_local_err.append(total.GetParError(2)+0.05*total.GetParameter(2))


        int_fg_local.append(g_gaus.Integral(total.GetParameter(1)-sigma*total.GetParameter(2),total.GetParameter(1)+sigma*total.GetParameter(2)))
        int_bg_local.append(g_pol.Integral(total.GetParameter(1)-sigma*total.GetParameter(2),total.GetParameter(1)+sigma*total.GetParameter(2)))
            
        #m2_pos.Draw("same") m2_neg.Draw("same")

        c1.SaveAs( "output/dynamic_delta_fits/" + m2_name + ".png" )

    return [Mean_local, Mean_local_err, Sigma_local, Sigma_local_err, int_fg_local, int_bg_local]


In [28]:
def emc_centr_fitter(InHist, layers, seed=1, sigma=2, fits_mean=0, fits_sigma=0):
    out_hists = []
    for icentr in range(N_centr):
        out_hists1 = []
        for ilayer in layers:
            out_hists1.append(emc_fitter(InHist[seed][icentr][iOption0+iOption1*2], ilayer, seed, icentr, sigma, fits_mean, fits_sigma))
        out_hists.append(out_hists1)
    return out_hists

In [29]:
import multiprocess as mp

Ntr = N_files

pool = mp.Pool(Ntr)
output_array = pool.starmap(emc_centr_fitter, [(hists_emc_read, [0,1,2,3,4,5], i, isigma) for i in range(Ntr)])
pool.close()

In [30]:
Means, Mean_errs, Sigmas, Sigma_errs, Int_FG, Int_BG = [], [], [], [], [], []
for ifile in range(Ntr):
    Means1, Mean_errs1, Sigmas1, Sigma_errs1, Int_FG1, Int_BG1 = [], [], [], [], [], []
    for icentr in range(N_centr):
        Means0, Mean_errs0, Sigmas0, Sigma_errs0, Int_FG0, Int_BG0 = [], [], [], [], [], []
        for ilayer in range(N_layers):
            Means0.append(output_array[ifile][icentr][ilayer][0])
            Mean_errs0.append(output_array[ifile][icentr][ilayer][1])
            Sigmas0.append(output_array[ifile][icentr][ilayer][2])
            Sigma_errs0.append(output_array[ifile][icentr][ilayer][3])
            Int_FG0.append(output_array[ifile][icentr][ilayer][4])
            Int_BG0.append(output_array[ifile][icentr][ilayer][5])
        Means1.append(Means0)
        Mean_errs1.append(Mean_errs0)
        Sigmas1.append(Sigmas0)
        Sigma_errs1.append(Sigma_errs0)
        Int_FG1.append(Int_FG0)
        Int_BG1.append(Int_BG0)
    Means.append(Means1)
    Mean_errs.append(Mean_errs1)
    Sigmas.append(Sigmas1)
    Sigma_errs.append(Sigma_errs1)
    Int_FG.append(Int_FG1)
    Int_BG.append(Int_BG1)

In [31]:
types = ["emc_r_dphi_","emc_min_r_dphi_","emc_cnt_dphi_","emc_r_dz_","emc_min_r_dz_","emc_cnt_dz_"]
colors=[1,2,4,6,root.kGreen+2]
fit_functions = []
for ifile in range(N_files):
    fit_functions0 = []
    for icentr in range(N_centr):
        fit_functions1 = []
        for ilayer in range(N_layers):
            fit_functions1.append(root.TF1(f"fit_functions_sigma_{ifile}_{ilayer}_{icentr}","[0]+[1]*exp([2]*x)",0.5,5.0))
            fit_functions1[-1].SetParameter(0,0.0015)
            fit_functions1[-1].SetParameter(1,0.02)
            fit_functions1[-1].SetParameter(2,-2)
            fit_functions1[-1].SetParLimits(2,-8.126,-0.023)
            if icentr<2 : fit_functions1[-1].SetParLimits(2,-5.5+0.5,-4.4-0.5)
            elif icentr<4 : fit_functions1[-1].SetParLimits(2,-3.0+0.45,-2.0-0.45)
            else : fit_functions1[-1].SetParLimits(2,-3.0+0.45,-2.0-0.45)
            fit_functions1[-1].SetParLimits(0,0.001,0.002)
            fit_functions1[-1].SetLineColor(colors[ilayer%4])
        fit_functions0.append(fit_functions1)
    fit_functions.append(fit_functions0)

In [32]:

print(Sigmas[0][0][0],Sigmas[0][3][0])
for ifile in range(N_files):
    h2=[]
    c3 = root.TCanvas(f"c3{ifile}",f"c3{ifile}",1500,900)
    c3.Divide(3,2)
    g_sigma = [[[[]]*N_layers]*N_centr]*N_files
    min, max = 0., 0.01
    if iOption1==1: max, min = 4.99, 0.01
    names = ["emc_r_dphi","emc_min_r_dphi","emc_cnt_dphi","emc_r_dz","emc_min_r_dz","emc_cnt_dz"]
    legends =[]
    for icentr in range(N_centr):
        c3.cd(icentr+1)
        h2.append(Format_Pad_old( 0.05, 5, min, max, "p_{T} (GeV/#it{c})", "Sigmas", 1.1, 1.1, 0.06, 0.05, "",0.15,0.15,0.,0.1))
        h2[-1].Draw()
        legends.append(root.TLegend(0.55,0.4,0.85,0.9,centralities[icentr]))
        legends[-1].SetFillColorAlpha(0,0)
        legends[-1].SetLineWidth(0)
        for ilayer in range(0,N_layers):
            g_sigma[ifile][icentr][ilayer].append(root.TGraphErrors(N_pt, pt, Sigmas[ifile][icentr][ilayer],pt_err,Sigma_errs[ifile][icentr][ilayer]))

            Format_Graph(g_sigma[ifile][icentr][ilayer][-1],20+int(ilayer/4),1, colors[ilayer%4], 3, colors[ilayer%4],1,1)
            g_sigma[ifile][icentr][ilayer][-1].Fit(fit_functions[ifile][icentr][ilayer],"Q","")
            g_sigma[ifile][icentr][ilayer][-1].Draw('P')
            legends[-1].AddEntry(g_sigma[ifile][icentr][ilayer][-1],f"layer {ilayer}","p")

        legends[-1].Draw()
    c3.Draw()
    c3.SaveAs(f"output/Sigmas/sigma_{ifile}{iOption0}{iOption1}.png")

array('d', [0.004466159874636573, 0.0030604668828061094, 0.0025660103299196295, 0.002285864001263618, 0.0021235646969772564, 0.0020122335326794444, 0.002027855632436842, 0.001980038739041637, 0.0018336135371124374, 0.0020303306961409846, 0.0013545618311360534, 0.012]) array('d', [0.007021358387034535, 0.0057033923010887146, 0.004575615872751091, 0.0038479390883648934, 0.0032883690691982953, 0.0031351621428835066, 0.002806865864807398, 0.0027791604757434656, 0.0024002192202872007, 0.002534594521351639, 0.012, 0.012])


In [33]:
print(f"params[{N_files}][{N_centr}][{N_layers}][3]="+"{")
for fit_functions_ifile in fit_functions:
    print("{")
    for fit_functions_icentr in fit_functions_ifile:
        print("{")
        for fits in fit_functions_icentr:
            print("{"+f"{round(fits.GetParameter(0),6)}, {round(fits.GetParameter(1),6)}, {round(fits.GetParameter(2),6)}"+"},")
        print("},") 
    print("},")      
print("};")        

params[1][6][6][3]={
{
{
{0.001994, 0.006032, -5.0},
{0.001792, 0.007533, -4.9},
{0.001754, 0.006984, -4.9},
{0.001787, 0.007025, -5.0},
{0.001782, 0.007374, -5.0},
{0.001976, 0.006351, -5.0},
},
{
{0.002, 0.006957, -5.0},
{0.001816, 0.008151, -4.900001},
{0.001839, 0.007737, -4.9},
{0.0019, 0.007491, -5.0},
{0.001901, 0.007536, -4.9},
{0.002, 0.005546, -4.9},
},
{
{0.002, 0.011558, -2.52906},
{0.001881, 0.01075, -2.45},
{0.001796, 0.01027, -2.45},
{0.00179, 0.009789, -2.45},
{0.001817, 0.009461, -2.494107},
{0.002, 0.008426, -2.55},
},
{
{0.002, 0.008139, -2.55},
{0.001773, 0.009259, -2.498868},
{0.001858, 0.009472, -2.55},
{0.001717, 0.010257, -2.45},
{0.001881, 0.010639, -2.549998},
{0.002, 0.010609, -2.55},
},
{
{0.002, 0.004103, -2.45},
{0.001759, 0.003646, -2.45},
{0.001281, 0.003987, -2.45},
{0.001197, 0.00377, -2.55},
{0.001183, 0.003685, -2.55},
{0.001371, 0.003182, -2.55},
},
{
{0.001716, 0.001944, -2.55},
{0.001153, 0.00384, -2.55},
{0.001231, 0.003729, -2.55},
{0.001391, 0.

In [34]:
colors=[1,2,4,6,root.kGreen+2]
fit_mean_functions = []
for ifile in range(N_files):
    fit_functions0 = []
    for icentr in range(N_centr):
        fit_functions1 = []
        for ilayer in range(N_layers):
            fit_functions1.append(root.TF1(f"fit_functions_mean_{ifile}_{ilayer}_"+centralities[icentr],"[0]",0.5,5.0))
            fit_functions1[-1].SetParameter(0,0.029)
            fit_functions1[-1].SetLineColor(colors[ilayer%4])
        fit_functions0.append(fit_functions1)
    fit_mean_functions.append(fit_functions0)

In [35]:
for ifile in range(N_files):
    h2=[]
    c3 = root.TCanvas(f"c3",f"c3",500*3,450*2)
    c3.Divide(3,2)
    g_sigma = [[[[]]*N_layers]*N_centr]*N_files
    min, max = -0.1, 0.1
    if iOption1==1: max, min = -1, 1
    names = ["emc_r_dphi","emc_min_r_dphi","emc_cnt_dphi","emc_r_dz","emc_min_r_dz","emc_cnt_dz"]
    legends =[]
    for icentr in range(N_centr):
        c3.cd(icentr+1)
        h2.append(Format_Pad_old( 0.05, 5, min, max, "p_{T} (GeV/#it{c})", "Means", 1.1, 1.1, 0.06, 0.05, "",0.15,0.15,0.,0.1))
        h2[-1].Draw()
        legends.append(root.TLegend(0.55,0.4,0.85,0.9,centralities[icentr]))
        legends[-1].SetFillColorAlpha(0,0)
        legends[-1].SetLineWidth(0)
        for ilayer in range(0,N_layers):

            g_sigma[ifile][icentr][ilayer].append(root.TGraphErrors(N_pt, pt, Means[ifile][icentr][ilayer],pt_err,Mean_errs[ifile][icentr][ilayer]))
            
            Format_Graph(g_sigma[ifile][icentr][ilayer][-1],20+int(ilayer/4),1, colors[ilayer%4], 3, colors[ilayer%4],1,1)
            g_sigma[ifile][icentr][ilayer][-1].Fit(fit_mean_functions[ifile][icentr][ilayer],"Q","")
            g_sigma[ifile][icentr][ilayer][-1].Draw('P')
            legends[-1].AddEntry(g_sigma[ifile][icentr][ilayer][-1],f"layer {ilayer}","p")

        legends[-1].Draw()
    c3.Draw()
    c3.SaveAs(f"output/Sigmas/mean_{ifile}{iOption0}{iOption1}.png")

In [36]:
print(f"params[{N_files}][{N_centr}][{N_layers}]="+"{")
for fit_functions_file in fit_mean_functions:
    print("{")
    for fit_functions_syst in fit_functions_file:
        print("{"+f"{round(fit_functions_syst[0].GetParameter(0),6)}, {round(fit_functions_syst[1].GetParameter(0),6)},{round(fit_functions_syst[2].GetParameter(0),6)}, {round(fit_functions_syst[3].GetParameter(0),6)}, {round(fit_functions_syst[4].GetParameter(0),6)}, {round(fit_functions_syst[5].GetParameter(0),6)}"+"},")
    print("},")
print("};")

params[1][6][6]={
{
{-0.010306, -0.006074,-0.001918, 0.00224, 0.006361, 0.010339},
{-0.008304, -0.004275,-0.000186, 0.003965, 0.008103, 0.012378},
{-0.011758, -0.00728,-0.002619, 0.001666, 0.005881, 0.009628},
{-0.007635, -0.003924,0.000316, 0.004593, 0.009263, 0.013791},
{-0.012672, -0.007632,0.027334, 0.001999, 0.030688, 0.010632},
{-0.008648, -0.004432,1.8e-05, 0.004706, 0.009626, 0.014506},
},
};


In [37]:
dphi = array( 'd' )
dphi_err = array( 'd' )
dynamic_dphi = []
dynamic_dphi_err = []
for ilayer in range(N_layers):
    dphi.append( round(0.002*layer_bins[2*ilayer]-0.001-0.1,5))
    dphi_err.append(0.001/2)

for ifile in range(N_files):
    dynamic_dphi_ifile = []
    dynamic_dphi_err_ifile = []
    for icentr in range(N_centr):
        dynamic_dphi_icentr = array( 'd' )
        dynamic_dphi_err_icentr = array( 'd' )
        for ilayer in range(N_layers):
            dynamic_dphi_icentr.append(fit_mean_functions[ifile][icentr][ilayer].GetParameter(0))
            dynamic_dphi_err_icentr.append(fit_mean_functions[ifile][icentr][ilayer].GetParError(0))
        dynamic_dphi_ifile.append(dynamic_dphi_icentr)
        dynamic_dphi_err_ifile.append(dynamic_dphi_err_icentr)
    dynamic_dphi.append(dynamic_dphi_ifile)
    dynamic_dphi_err.append(dynamic_dphi_err_ifile)

print("dphi = ", dphi)
print(dynamic_dphi)

dphi =  array('d', [-0.009, -0.005, -0.001, 0.003, 0.007, 0.011])
[[array('d', [-0.01030608460854968, -0.006074379516177033, -0.0019175658629733208, 0.0022401069787497396, 0.006361052486336633, 0.01033949106386783]), array('d', [-0.008304201929135852, -0.004275354999679109, -0.00018564687138731065, 0.003965088077810486, 0.008102827897060096, 0.012377637354327821]), array('d', [-0.011758020695723816, -0.007279799022894159, -0.002619412813733759, 0.0016655427797032593, 0.005880730320030784, 0.009628120060724443]), array('d', [-0.0076352241317115856, -0.003923878389834042, 0.0003163046905659411, 0.0045925775593423784, 0.009262912370697849, 0.013791021725689658]), array('d', [-0.012672472271070715, -0.007631569738726123, 0.027333985468680884, 0.0019990189644579007, 0.03068834677725365, 0.01063249677540477]), array('d', [-0.008647805062728334, -0.004432066149686182, 1.7831669544866868e-05, 0.00470642422942218, 0.009625515255436114, 0.014506251582231882])]]


In [38]:
for ifile in range(N_files):
    h3=[]
    c4 = root.TCanvas(f"c4",f"c4",600,450)
    g_sigma = [[[[]]*N_layers]*N_centr]*N_files
    min, max = -0.05, 0.05
    if iOption1==1: max, min = -1, 1
    names = ["emc_r_dphi","emc_min_r_dphi","emc_cnt_dphi","emc_r_dz","emc_min_r_dz","emc_cnt_dz"]
    legends =[]
    for ilayer in range(1):
        h3.append(Format_Pad_old( -0.02, 0.05, min, max, "p_{T} (GeV/#it{c})", "Means", 1.1, 1.1, 0.06, 0.05, "",0.15,0.15,0.,0.1))
        h3[-1].Draw()
        legends.append(root.TLegend(0.55,0.4,0.85,0.9))
        legends[-1].SetFillColorAlpha(0,0)
        legends[-1].SetLineWidth(0)
        for icentr in range(0,N_centr):

            g_sigma[ifile][icentr][ilayer].append(root.TGraphErrors(N_layers, dphi, dynamic_dphi[ifile][icentr],dphi_err,dynamic_dphi_err[ifile][icentr]))
            
            Format_Graph(g_sigma[ifile][icentr][ilayer][-1],20+int(icentr/4),1, colors[icentr%4], 3, colors[icentr%4],1,1)
            g_sigma[ifile][icentr][ilayer][-1].Fit("pol1","","")
            g_sigma[ifile][icentr][ilayer][-1].Draw('sameP')
            legends[-1].AddEntry(g_sigma[ifile][icentr][ilayer][-1],f"layer {icentr}","p")

        legends[-1].Draw()
    c4.Draw()
    c4.SaveAs(f"output/Sigmas/mean_{ifile}{iOption0}{iOption1}.png")

 FCN=0.0963808 FROM MIGRAD    STATUS=CONVERGED      40 CALLS          41 TOTAL
                     EDM=1.87386e-07    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  p0          -9.26439e-04   2.14092e-04   1.08281e-07  -1.02116e+00
   2  p1           1.03362e+00   3.10320e-02   1.56840e-05   1.74359e-02
 FCN=0.0940759 FROM MIGRAD    STATUS=CONVERGED      48 CALLS          49 TOTAL
                     EDM=5.37763e-09    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  p0           9.13004e-04   2.14169e-04   1.08223e-07   4.68743e-01
   2  p1           1.03362e+00   3.10426e-02   1.56858e-05   1.43040e-03
 FCN=1.33188 FROM MIGRAD    STATUS=CONVERGED      43 CALLS          44 TOTAL
                     EDM=

In [39]:
sigma_dynamic_dphi = []
sigma_dynamic_dphi_err = []


for ifile in range(N_files):
    dynamic_dphi_ifile = []
    dynamic_dphi_err_ifile = []
    for icentr in range(N_centr):
        dynamic_dphi_icentr = array( 'd' )
        dynamic_dphi_err_icentr = array( 'd' )
        for ilayer in range(N_layers):
            dynamic_dphi_icentr.append(fit_functions[ifile][icentr][ilayer].GetParameter(1)+fit_functions[ifile][icentr][ilayer].GetParameter(0))
            dynamic_dphi_err_icentr.append(fit_functions[ifile][icentr][ilayer].GetParError(1)+fit_functions[ifile][icentr][ilayer].GetParameter(0))
        dynamic_dphi_ifile.append(dynamic_dphi_icentr)
        dynamic_dphi_err_ifile.append(dynamic_dphi_err_icentr)
    sigma_dynamic_dphi.append(dynamic_dphi_ifile)
    sigma_dynamic_dphi_err.append(dynamic_dphi_err_ifile)

print(sigma_dynamic_dphi)

[[array('d', [0.00802642777623851, 0.009324609701144407, 0.008737438517205083, 0.008811973102004069, 0.009155842795962722, 0.00832680246604929]), array('d', [0.008957491962381383, 0.00996689325503796, 0.009576022837037653, 0.00939173076716595, 0.009437111153901293, 0.007546032279833709]), array('d', [0.013557724852257057, 0.012631178958772558, 0.01206586052922935, 0.011579129160704195, 0.011278130903759401, 0.010426097948379565]), array('d', [0.010139021304835575, 0.01103219006827837, 0.011330006183272335, 0.011974949708056848, 0.012520578159197762, 0.012609277539124879]), array('d', [0.006103314250559809, 0.00540483475410972, 0.00526750571026976, 0.0049671310115895605, 0.004867840119547273, 0.004552352720869804]), array('d', [0.003659554101831778, 0.004992571190503333, 0.004959982405728758, 0.00522658937593432, 0.005811219671341494, 0.006256245642121103])]]


In [40]:
for ifile in range(N_files):
    h3=[]
    c4 = root.TCanvas(f"c4",f"c4",600,450)
    g_sigma = [[[[]]*N_layers]*N_centr]*N_files
    min, max = -0.05, 0.05
    if iOption1==1: max, min = -1, 1
    names = ["emc_r_dphi","emc_min_r_dphi","emc_cnt_dphi","emc_r_dz","emc_min_r_dz","emc_cnt_dz"]
    legends =[]
    for ilayer in range(1):
        h3.append(Format_Pad_old( -0.02, 0.05, min, max, "p_{T} (GeV/#it{c})", "Means", 1.1, 1.1, 0.06, 0.05, "",0.15,0.15,0.,0.1))
        h3[-1].Draw()
        legends.append(root.TLegend(0.55,0.4,0.85,0.9))
        legends[-1].SetFillColorAlpha(0,0)
        legends[-1].SetLineWidth(0)
        for icentr in range(0,N_centr):

            g_sigma[ifile][icentr][ilayer].append(root.TGraphErrors(N_layers, dphi, sigma_dynamic_dphi[ifile][icentr],dphi_err,sigma_dynamic_dphi_err[ifile][icentr]))
            
            Format_Graph(g_sigma[ifile][icentr][ilayer][-1],20+int(icentr/4),1, colors[icentr%4], 3, colors[icentr%4],1,1)
            g_sigma[ifile][icentr][ilayer][-1].Fit("pol2","","")
            g_sigma[ifile][icentr][ilayer][-1].Draw('sameP')
            legends[-1].AddEntry(g_sigma[ifile][icentr][ilayer][-1],f"layer {icentr}","p")

        legends[-1].Draw()
    c4.Draw()
    c4.SaveAs(f"output/Sigmas/mean_{ifile}{iOption0}{iOption1}.png")

 FCN=0.0677146 FROM MIGRAD    STATUS=CONVERGED      87 CALLS          88 TOTAL
                     EDM=6.75906e-10    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  p0           9.06585e-03   1.85047e-03   6.14589e-07   1.72922e-02
   2  p1           2.36476e-02   1.89424e-01   8.92641e-05  -9.00269e-05
   3  p2          -7.62004e+00   3.05182e+01   9.68636e-03  -1.49201e-07
 FCN=0.0545371 FROM MIGRAD    STATUS=CONVERGED      58 CALLS          59 TOTAL
                     EDM=1.35669e-07    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  p0           9.85798e-03   1.96306e-03   6.37897e-07  -2.61196e-01
   2  p1          -3.50770e-02   1.93104e-01   9.12740e-05  -1.94123e-03
   3  p2          -1.43197e+0